<a href="https://colab.research.google.com/github/kamini8888/ai/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plot
from glob import glob

In [60]:
IMAGESHAPE = [224, 224, 3]
training_data = '/content/drive/MyDrive/kaggle/imagess/train'
testing_data = '/content/drive/MyDrive/kaggle/imagess/test'

In [61]:
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)


In [62]:
for each_layer in vgg_model.layers:
	each_layer.trainable = False


In [64]:
classes = glob('/content/drive/MyDrive/kaggle/imagess/train/*')


In [65]:
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)


In [66]:
final_model = Model(inputs=vgg_model.input, outputs=prediction)
final_model.summary()


Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [68]:
final_model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)


In [69]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
								shear_range = 0.2,
								zoom_range = 0.2,
								horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)


In [71]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/kaggle/imagess/train',
												target_size = (224, 224),
												batch_size = 4,
												class_mode = 'categorical')


Found 481 images belonging to 2 classes.


In [72]:
test_set = testing_datagen.flow_from_directory('/content/drive/MyDrive/kaggle/imagess/test',
											target_size = (224, 224),
											batch_size = 4,
											class_mode = 'categorical')


Found 446 images belonging to 2 classes.


In [73]:
fitted_model = final_model.fit_generator(
training_set,
validation_data=test_set,
epochs=5,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)


<ipython-input-73-b2adf78dfb61>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fitted_model = final_model.fit_generator(


Epoch 1/5
121/121 [==============================] - 547s 4s/step - loss: 0.3203 - accuracy: 0.8857 - val_loss: 0.5162 - val_accuracy: 0.8161
Epoch 2/5
121/121 [==============================] - 527s 4s/step - loss: 0.1539 - accuracy: 0.9418 - val_loss: 0.5604 - val_accuracy: 0.8274
Epoch 3/5
121/121 [==============================] - 530s 4s/step - loss: 0.1137 - accuracy: 0.9584 - val_loss: 0.4007 - val_accuracy: 0.8722
Epoch 4/5
121/121 [==============================] - 541s 4s/step - loss: 0.1171 - accuracy: 0.9605 - val_loss: 0.6153 - val_accuracy: 0.8318
Epoch 5/5
121/121 [==============================] - 527s 4s/step - loss: 0.1471 - accuracy: 0.9480 - val_loss: 1.9257 - val_accuracy: 0.6547


In [74]:
final_model.save('our_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [76]:
from keras.models import load_model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
import numpy as np

# Load the model
model = load_model('our_model.h5')

# Load and preprocess the image without using keras_preprocessing
img_path = '/content/drive/MyDrive/kaggle/imagess/val/NORMAL/NORMAL2-IM-1427-0001.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Make predictions
prediction = model.predict(img_array)

# Print the result
if prediction[0][0] > prediction[0][1]:
    print('Person is safe.')
else:
    print('Person is affected with Pneumonia.')

print(f'Predictions: {prediction}')


1/1 [==============================] - 1s 643ms/step
Person is safe.
Predictions: [[1. 0.]]


In [98]:
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
import numpy as np

# Load the model
model = load_model('our_model.h5')

# Load and preprocess the image without using keras_preprocessing
img_path = '/content/drive/MyDrive/kaggle/chest_xray/train/PNEUMONIA/person1000_bacteria_2931.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_data = preprocess_input(img_array)

# Make predictions
prediction = model.predict(img_data)

# Print the result
if prediction[0][0] > prediction[0][1]:
    print('Person is safe.')
else:
    print('Person is affected with Pneumonia.')

print(f'Predictions: {prediction}')


1/1 [==============================] - 1s 1s/step
Person is affected with Pneumonia.
Predictions: [[5.3103685e-25 1.0000000e+00]]


In [99]:
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
import numpy as np

# Load the model
model = load_model('our_model.h5')

# Load and preprocess the image without using keras_preprocessing
img_path = '/content/drive/MyDrive/kaggle/imagess/val/NORMAL/NORMAL2-IM-1427-0001.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_data = preprocess_input(img_array)

# Make predictions
prediction = model.predict(img_data)

# Print the result
if prediction[0][0] > prediction[0][1]:
    print('Person is safe.')
else:
    print('Person is affected with Pneumonia.')

print(f'Predictions: {prediction}')


1/1 [==============================] - 1s 1s/step
Person is safe.
Predictions: [[1. 0.]]
